In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys

from scipy.stats import spearmanr
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import PredefinedSplit, GridSearchCV
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_lo_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/lo/kcnh2/train_1.csv', index_col=0)
test = pd.read_csv('../../../../data/lo/kcnh2/test_1.csv', index_col=0)

test

,smiles,value,cluster
0,C=C(C)COc1ccccc1CN1CCC2(CC1)CCN(C(=O)c1ccncc1)CC2,5.794709,20
1,C=CCOC[C@H]1CC[C@@H](N2CC(NC(=O)CNc3nn(C)c4ccc...,5.300943,31
2,C=CCO[C@H]1CC[C@@H](N2CC(NC(=O)CNc3n[nH]c4ccc(...,5.130710,31
3,CC(=O)N1CCC(C2N[C@@H](c3nc(-c4ccccc4)c[nH]3)Cc...,5.008730,34
4,CC(=O)NC1CCN(CCc2ccc(Oc3nc4ccccc4s3)cc2)CC1,5.045709,12
...,...,...,...
401,c1ccc(-c2c[nH]c([C@H]3Cc4c([nH]c5ccccc45)[C@@H...,6.419075,34
402,c1ccc(-c2c[nH]c([C@H]3Cc4c([nH]c5ccccc45)[C@H]...,6.136083,34
403,c1ccc(-c2ccc(-c3c[nH]c([C@H]4Cc5c([nH]c6ccccc5...,7.744727,34
404,c1ccc(CCCNCCN(c2ccccc2)c2ccccc2)cc1,6.217567,11


In [4]:
def spearman_scorer(clf, X, y):
    if len(X) == len(train):
        y_pred = clf.predict(X)
        metrics = get_lo_metrics(train, y_pred)
        return metrics['spearman']
    elif len(X) == len(test):
        y_pred = clf.predict(X)
        metrics = get_lo_metrics(test, y_pred)
        return metrics['spearman']
    else:
        raise ValueError


In [5]:
from scipy.spatial.distance import jaccard


def run_knn_gridsearch_tanimoto(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
        'n_neighbors': [15, 20, 25],
        'weights': ['uniform', 'distance'],
    }
    knn = KNeighborsRegressor(metric=jaccard)

    grid_search = GridSearchCV(knn, params, cv=pds, refit=False, scoring=spearman_scorer, verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    print(best_params)
    knn = KNeighborsRegressor(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'], metric=jaccard)
    knn.fit(train_fps, train['value'])

    test_preds = knn.predict(test_fps)
    test_metrics = get_lo_metrics(test, test_preds)
    return test_metrics


In [6]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

test_metrics = run_knn_gridsearch_tanimoto(train_maccs_fps, test_maccs_fps)
print(test_metrics)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV 1/1] END ...n_neighbors=15, weights=uniform;, score=0.058 total time=  13.0s
[CV 1/1] END ..n_neighbors=15, weights=distance;, score=0.049 total time=  13.0s
[CV 1/1] END ..n_neighbors=20, weights=uniform;, score=-0.084 total time=  13.0s
[CV 1/1] END .n_neighbors=20, weights=distance;, score=-0.014 total time=  13.0s
[CV 1/1] END ..n_neighbors=25, weights=uniform;, score=-0.006 total time=  12.9s
[CV 1/1] END ..n_neighbors=25, weights=distance;, score=0.068 total time=  12.8s
{'n_neighbors': 25, 'weights': 'distance'}
{'r2': -0.8860536492975095, 'spearman': 0.0675777383089241, 'mae': 0.9550683001652004}


# Final Evaluation

In [7]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_maccs_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in test_mols]

    knn = KNeighborsRegressor(n_neighbors=25, weights='distance', metric=jaccard)
    knn.fit(train_maccs_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = knn.predict(train_maccs_fps)

    test_result = test.copy()
    test_result['preds'] = knn.predict(test_maccs_fps)

    return train_result, test_result


In [8]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/lo/kcnh2/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/lo/kcnh2/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/lo/kcnh2/knn_maccs/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/lo/kcnh2/knn_maccs/test_{i}.csv')
